# Tarea 4

In [2]:
# Importar librerías 

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis 
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis 
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, precision_score

## Ejercicio 3

In [4]:
# Importar y limpiar datos
ejemplo = pd.read_csv("Ejemplo_AD.csv", header=None, sep=';')
ejemplo = ejemplo.drop(columns=7)

# Renombrar las columnas
n = ejemplo.shape[1]
nombres_columnas = ['id'] + [f'x_{i}' for i in range(1, n-1)] + ['grupo']
ejemplo.columns = nombres_columnas

# Convertir las columnas numéricas a float si es necesario
for col in ejemplo.columns[1:-1]:
    ejemplo[col] = pd.to_numeric(ejemplo[col], errors='coerce')

# Mostrar primeras filas
ejemplo.head()

,id,x_1,x_2,x_3,x_4,x_5,grupo
0,A1,9.0,4.6,2.0,0.1,25.8,A
1,B1,4.0,3.3,0.4,0.6,32.4,B
2,C1,1.4,1.0,1.1,0.5,23.5,C
3,A2,10.0,6.7,3.9,0.2,15.7,A
4,B2,8.2,6.2,2.1,0.2,17.0,B


In [5]:
def calcular_centro_gravedad(df, clase):
    """
    Calcula el centro de gravedad g_s para una clase dada.
    
    Parámetros:
        df: DataFrame con columnas ['id', x_1, ..., x_p, 'grupo']
        clase: str, nombre de la clase (por ejemplo 'A')
    
    Retorna:
        Vector g_s como Series de pandas
    """
    # Filtrar por clase
    grupo_df = df[df['grupo'] == clase]
    
    # Extraer solo las variables numéricas
    x = grupo_df.drop(columns=['id', 'grupo'])

    # Calcular el centro de gravedad (promedio por columnas porque se asume p_i = 1/n)
    g_s = x.mean(axis=0)

    return pd.Series(g_s, index=x.columns, name=f'g_{clase}')

In [6]:
def calcular_cov_total(df):
    """
    Calcula la matriz V = X^T D X sin centrar los datos.
    
    Parámetros:
        df: DataFrame con columnas ['id', x_1, ..., x_p, 'grupo']
    
    Retorna:
        Matriz V como DataFrame p x p
    """
    # Extraer variables numéricas de la matriz X
    X = df.drop(columns=['id', 'grupo']).to_numpy()

    # Número de observaciones
    n = len(X)

    # D = diag(p_i), donde p_i = 1/n
    p_i = np.ones(n) / n
    D = np.diag(p_i)

    # Calcular V = X^T D X
    V = X.T @ D @ X

    # Nombres de columnas para resultado
    nombres_columnas = df.drop(columns=['id', 'grupo']).columns
    
    return pd.DataFrame(V, index=nombres_columnas, columns=nombres_columnas)

In [7]:
def calcular_cov_inter(df):
    """
    Calcula la matriz de covarianza inter-clase V_B.
    
    Parámetros:
        df: DataFrame con columnas ['id', x_1, ..., x_p, 'grupo']
    
    Retorna:
        Matriz V_B como DataFrame p x p
    """
    # Extraer nombres de clases y columnas numéricas
    clases = df['grupo'].unique()
    columnas = df.drop(columns=['id', 'grupo']).columns
    p = len(columnas)
    
    n = len(df)  # total de observaciones
    V_B = np.zeros((p, p))  # matriz acumuladora

    for s in clases:
        grupo_s = df[df['grupo'] == s]
        n_s = len(grupo_s)
        q_s = n_s / n  # peso total de la clase
        g_s = calcular_centro_gravedad(df, s).to_numpy().reshape(-1, 1)  # columna
        
        # g_s @ g_s.T es el producto exterior
        V_B += q_s * (g_s @ g_s.T)

    return pd.DataFrame(V_B, index=columnas, columns=columnas)

In [8]:
def calcular_cov_intra(df):
    """
    Calcula la matriz de covarianza intra-clase V_W (versión vectorizada).
    
    Parámetros:
        df: DataFrame con columnas ['id', x_1, ..., x_p, 'grupo']
    
    Retorna:
        Matriz V_W como DataFrame p x p
    """
    # Extraer nombres de clases y columnas numéricas
    clases = df['grupo'].unique()
    columnas = df.drop(columns=['id', 'grupo']).columns
    p = len(columnas)

    n = len(df)
    p_i = 1 / n
    V_W = np.zeros((p, p), dtype=np.float64)  # matriz acumuladora

    for s in clases:
        grupo_s = df[df['grupo'] == s]
        X_s = grupo_s[columnas].to_numpy(dtype=np.float64)  # n_s x p

        # Centro de gravedad del grupo s
        g_s = calcular_centro_gravedad(df, s).to_numpy(dtype=np.float64).flatten()

        # Resta vectorial fila a fila
        resta = X_s - g_s  #

        # Acumular producto matricial de las diferencias
        V_W += p_i * (resta.T @ resta)

    return pd.DataFrame(V_W, index=columnas, columns=columnas)

### Resultados

In [10]:
# Cálculo g_A
g_A = calcular_centro_gravedad(ejemplo, 'A')
g_A

x_1    10.90
x_2     6.59
x_3     3.29
x_4     0.36
x_5    21.34
Name: g_A, dtype: float64

In [11]:
# Cálculo g_B
g_B = calcular_centro_gravedad(ejemplo, 'B')
g_B

x_1     6.70
x_2     5.84
x_3     1.74
x_4     0.96
x_5    22.22
Name: g_B, dtype: float64

In [12]:
# Cálculo g_C
g_C = calcular_centro_gravedad(ejemplo, 'C')
g_C

x_1     4.04
x_2     4.81
x_3     0.82
x_4     0.81
x_5    21.66
Name: g_C, dtype: float64

In [13]:
# Matriz covarianza total V
V = calcular_cov_total(ejemplo)
V

,x_1,x_2,x_3,x_4,x_5
x_1,62.924667,42.562333,17.394667,4.510000,153.736667
x_2,42.562333,43.864000,13.611000,4.551667,120.411000
x_3,17.394667,13.611000,5.718333,1.267000,39.976333
x_4,4.510000,4.551667,1.267000,1.097000,16.453333
x_5,153.736667,120.411000,39.976333,16.453333,489.894667


In [14]:
# Matriz covarianza inter-clase V_B
V_B = calcular_cov_inter(ejemplo)
V_B

,x_1,x_2,x_3,x_4,x_5
x_1,60.007200,43.463800,16.943933,4.5428,156.328800
x_2,43.463800,33.556600,11.928967,3.9583,124.860000
x_3,16.943933,11.928967,4.841367,1.1730,42.210867
x_4,4.542800,3.958300,1.173000,0.5691,15.519400
x_5,156.328800,124.860000,42.210867,15.5194,472.759867


In [15]:
# Matriz covarianza intra-clase V_W
V_W = calcular_cov_intra(ejemplo)
V_W

,x_1,x_2,x_3,x_4,x_5
x_1,2.917467,-0.901467,0.450733,-0.032800,-2.592133
x_2,-0.901467,10.307400,1.682033,0.593367,-4.449000
x_3,0.450733,1.682033,0.876967,0.094000,-2.234533
x_4,-0.032800,0.593367,0.094000,0.527900,0.933933
x_5,-2.592133,-4.449000,-2.234533,0.933933,17.134800


In [16]:
# Verificar V_B + V_W = V
ver = V_B + V_W
ver

,x_1,x_2,x_3,x_4,x_5
x_1,62.924667,42.562333,17.394667,4.510000,153.736667
x_2,42.562333,43.864000,13.611000,4.551667,120.411000
x_3,17.394667,13.611000,5.718333,1.267000,39.976333
x_4,4.510000,4.551667,1.267000,1.097000,16.453333
x_5,153.736667,120.411000,39.976333,16.453333,489.894667


## Ejercicio 5

### Inciso a)

In [19]:
# Importar datos
diabetes = pd.read_csv("diabetes.csv", sep=',')
diabetes.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


### Inciso b)

In [21]:
# Dividir los datos en X y y
X = diabetes.drop(columns='Outcome').to_numpy()
y = diabetes['Outcome'].to_numpy()

# Dividir los datos en train y test (75% entrenamiento, 25% prueba)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.25,
    random_state=42
)

In [22]:
# Aplicar modelo LDA
lda = LinearDiscriminantAnalysis()
lda.fit(X_train, y_train)
y_pred_lda = lda.predict(X_test)

# Matriz de confusión
conf_lda = confusion_matrix(y_test, y_pred_lda)
print("Matriz de confusión LDA:\n", conf_lda)

# Precisión global
precision_lda = precision_score(y_test, y_pred_lda, average='weighted')
print(f"Precisión global: {precision_lda:.2f}")

# Precisión por clase
print("Precision por clase:", precision_score(y_test, y_pred_lda, average=None))

Matriz de confusión LDA:
 [[96 27]
 [25 44]]
Precisión global: 0.73
Precision por clase: [0.79338843 0.61971831]


In [23]:
# Aplicar modelo QDA
qda = QuadraticDiscriminantAnalysis()
qda.fit(X_train, y_train)
y_pred_qda = qda.predict(X_test)

# Matriz de confusión
conf_qda = confusion_matrix(y_test, y_pred_qda)
print("Matriz de confusión QDA:\n", conf_qda)

# Precisión global
precision_qda = precision_score(y_test, y_pred_qda, average='weighted')
print(f"Precisión global: {precision_qda:.2f}")

# Precisión por clase
print("Precision por clase:", precision_score(y_test, y_pred_qda, average=None))

Matriz de confusión QDA:
 [[99 24]
 [21 48]]
Precisión global: 0.77
Precision por clase: [0.825      0.66666667]


In [24]:
# Aplicar modelo Bayes
bayes = GaussianNB()
bayes.fit(X_train, y_train)
y_pred_bayes = bayes.predict(X_test)

# Matriz de confusión
conf_bayes = confusion_matrix(y_test, y_pred_bayes)
print("Matriz de confusión Bayes:\n", conf_bayes)

# Precisión global
precision_bayes = precision_score(y_test, y_pred_bayes, average='weighted')
print(f"Precisión global: {precision_bayes:.2f}")

# Precisión por clase
print("Precision por clase:", precision_score(y_test, y_pred_bayes, average=None))

Matriz de confusión Bayes:
 [[94 29]
 [22 47]]
Precisión global: 0.74
Precision por clase: [0.81034483 0.61842105]


Los tres modelos obtuvieron resultados parecidos en sus predicciones. En general, según la matriz de confusión, suelen haber alrededor de 90 negativos verdaderos y 20 negativos falsos, lo cual es un buen indicio. Asimismo, suelen haber cerca del doble de positivos verdaderos que de falsos. Sin embargo, la cantidad de negativos/positivos falsos está elevada y esto se ve reflejado en las métricas de precisión, las cuales entre más cerca se encuentren de uno, mejor. Se puede resaltar el hecho de que para la primera clase la precisión es mayor que para la segunda clase. Por lo tanto, aunque 0.70 no es un número muy bajo, se puede mejorar.  

### Inciso c)

In [27]:
def evaluar_modelo(nombre, y_true, y_pred):
    precision_global = precision_score(y_test, y_pred, average='weighted')
    error_global = 1 - precision_global
    pp = precision_score(y_true, y_pred, pos_label=1)
    pn = precision_score(y_true, y_pred, pos_label=0)

    return {
        "Modelo": nombre,
        "Precisión global": round(precision_global, 4),
        "Error global": round(error_global, 4),
        "PP": round(pp, 4),
        "PN": round(pn, 4)
    }

In [28]:
resultados = []

# LDA
resultados.append(evaluar_modelo("LDA", y_test, y_pred_lda))

# QDA
resultados.append(evaluar_modelo("QDA", y_test, y_pred_qda))

# Naive Bayes
resultados.append(evaluar_modelo("Bayes", y_test, y_pred_bayes))

df_resultados = pd.DataFrame(resultados)
df_resultados

,Modelo,Precisión global,Error global,PP,PN
0,LDA,0.7310,0.2690,0.6197,0.7934
1,QDA,0.7681,0.2319,0.6667,0.8250
2,Bayes,0.7414,0.2586,0.6184,0.8103


### Inciso d)

In [30]:
# Imprimir columnas (variables predictoras)
diabetes.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')

In [31]:
# Seleccionar las primeras seis
col_sel = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']

# Crear X, y otra vez
X = diabetes[col_sel].to_numpy()
y = diabetes['Outcome'].to_numpy()

# Volver a dividir en train y set
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.25,
    random_state=42
)

# Entrenar y comparar modelos
resultados_nuevos = []

# LDA
lda = LinearDiscriminantAnalysis()
lda.fit(X_train, y_train)
y_pred_lda_nuevo = lda.predict(X_test)
resultados_nuevos.append(evaluar_modelo("LDA", y_test, y_pred_lda_nuevo))

# QDA
qda = QuadraticDiscriminantAnalysis()
qda.fit(X_train, y_train)
y_pred_qda_nuevo = qda.predict(X_test)
resultados_nuevos.append(evaluar_modelo("QDA", y_test, y_pred_qda_nuevo))

# Bayes
bayes = GaussianNB()
bayes.fit(X_train, y_train)
y_pred_bayes_nuevo = bayes.predict(X_test)
resultados_nuevos.append(evaluar_modelo("Bayes", y_test, y_pred_bayes_nuevo))

# Mostrar resultados nuevos
pd.DataFrame(resultados_nuevos)

,Modelo,Precisión global,Error global,PP,PN
0,LDA,0.7578,0.2422,0.6769,0.8031
1,QDA,0.7517,0.2483,0.6479,0.8099
2,Bayes,0.7485,0.2515,0.6567,0.8000


Comparando la tabla de resultados con la de resultados_nuevos, no se ve reflejada una mejora significativa en la predicción. En realidad, para los modelos QDA y Bayes, se observa una disminuición en la precisión global. Mientras que en LDA existe un leve aumento.